In [1]:
import urllib.request 
import os
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.linear_model import LinearRegression
import datetime

In [2]:
## Download and unzip COVID cases
temporary_folder = os.path.join(os.getcwd(), 'tmp')
if not os.path.exists(temporary_folder):
    os.makedirs(temporary_folder)
    
remote_file_name = "https://github.com/CSSEGISandData/COVID-19/archive/master.zip"
base_name = os.path.basename(remote_file_name)
local_file_name = os.path.join(temporary_folder, base_name)

urllib.request.urlretrieve(remote_file_name, local_file_name)

data_folder = os.path.join(os.getcwd(), 'world_data') 
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
    
with zipfile.ZipFile(local_file_name, 'r') as zip_ref:
    zip_ref.extractall(data_folder)
    
#!unzip master.zip

In [3]:
## Download world population
remote_file_name = "http://api.worldbank.org/v2/en/indicator/SP.POP.TOTL?downloadformat=excel"
world_population_file = os.path.join(temporary_folder, "world_population.xls")

# The following statement is throwing an error but the download is being performed
urllib.request.urlretrieve(remote_file_name, world_population_file) 

('/Users/wcunha69/Documents/projects/COVID-19/tmp/world_population.xls',
 <http.client.HTTPMessage at 0x1a238101d0>)

In [4]:
## Load world population DataFrame
world_population = pd.read_excel(world_population_file, sheet_name="Data", dtype=object, skiprows=3)
world_population = world_population[["Country Name", "Country Code", "2018"]]
world_population = world_population.rename(
    {"Country Name":"country","Country Code":"country_code", "2018":"population"}, 
    axis='columns') 

world_population['population'] = world_population['population'].fillna(0)
world_population['population'] = world_population['population'].astype('int')
#print(world_population)


In [5]:
## Load COVID cases 
covid_folder = os.path.join(data_folder, 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/')

li = []
for covid_file in os.listdir(covid_folder):
    if covid_file.find(".csv") > -1:
        df = pd.read_csv(os.path.join(covid_folder, covid_file), index_col=None, header=0)
        li.append(df)

covid_cases = pd.concat(li, axis=0, ignore_index=True)
covid_cases = covid_cases.rename(
    {
        "Province/State":"province_state",
        "Country/Region":"country", 
        "Last Update":"last_update",
        "Confirmed":"confirmed",
        "Deaths":"deaths",
        "Recovered":"recovered",
        "Latitude":"latitude",
        "Longitude":"longitude"
    }, axis='columns'
) 


#print(covid_cases[covid_cases["country"] == "Italy"])
#print(recent)
#covid = pd.read_csv(covid)
#print(covid)

In [6]:
## Adjusting country names and merging dataframes
world_population = world_population.replace("Korea, Rep.", "South Korea")
world_population = world_population.replace("Hong Kong SAR, China", "Hong Kong")
world_population = world_population.replace("Iran, Islamic Rep.", "Iran")
world_population = world_population.replace("Russian Federation", "Russia")
world_population = world_population.replace("Macao SAR, China", "Macau")
world_population = world_population.replace("Venezuela, RB", "Venezuela")
world_population = world_population.replace("Bahamas, The", "The Bahamas")
world_population = world_population.replace("Egypt, Arab Rep.", "Egypt")
world_population = world_population.replace("Slovak Republic", "Slovakia")
covid_cases = covid_cases.replace("Mainland China", "China")
covid_cases = covid_cases.replace("US", "United States")
covid_cases = covid_cases.replace("UK", "United Kingdom")
covid_cases = covid_cases.replace("Iran (Islamic Republic of)", "Iran")
covid_cases = covid_cases.replace("Russian Federation", "Russia")
covid_cases = covid_cases.replace("Republic of Korea", "South Korea")
covid_cases = covid_cases.replace("Hong Kong SAR", "Hong Kong")
covid_cases = covid_cases.replace("Viet Nam", "Vietnam")
covid_cases = covid_cases.replace("Macao SAR", "Macau")
covid_cases = covid_cases.replace("Korea, South", "South Korea")
covid_cases = covid_cases.replace("North Ireland", "United Kingdom")
covid_cases = covid_cases.replace("Republic of Ireland", "Ireland")

covid_cases_pop = pd.merge(covid_cases, world_population, how='left', on='country')

# The following line was used to check countries on population data set and not on cases and vice-versa
# print(((covid_cases_pop[covid_cases_pop['population'].isnull()])["country"]).drop_duplicates())

In [7]:
## Remove countries with no population 
covid_cases_pop['population'] = covid_cases_pop['population'].fillna(0)
covid_cases_pop = covid_cases_pop.loc[covid_cases_pop['population'] != 0]
covid_cases_pop['population'] = covid_cases_pop['population'].astype('int')
covid_cases_pop["last_update"] = pd.to_datetime(covid_cases_pop["last_update"])
covid_cases_pop["last_update"] = covid_cases_pop["last_update"].dt.date

covid_cases_aggr = covid_cases_pop[["country", "last_update", "population", "confirmed", "deaths"]]
#covid_cases_aggr["last_update"] = pd.to_datetime(covid_cases_aggr["last_update"])
#covid_cases_aggr["last_update"] = covid_cases_aggr["last_update"].dt.date
#print((covid_cases_aggr[covid_cases_aggr["country"] == "Italy"]).sort_values(by="last_update"))





In [8]:
## Aggregate by country, date and population
covid_cases_aggr = covid_cases_aggr.groupby(["country", "last_update", "population"]).sum()
covid_cases_aggr = covid_cases_aggr.reset_index()

#print((covid_cases_aggr[covid_cases_aggr["country"] == "Italy"]).sort_values(by="last_update"))

In [12]:
days_to_forecast = 10

#days_difference = covid_cases_aggr["last_update"].max() - covid_cases_aggr["last_update"].min()
#max_days = days_difference.days + 4

countries = covid_cases_aggr[covid_cases_aggr["country"].isin(["Brazil", "Italy", "United States", "South Korea"])]
countries = countries["country"].drop_duplicates()

#covid_cases_aggr
#countries = covid_cases_aggr["country"].drop_duplicates()

li = []

for country in countries:
    country_cases = covid_cases_aggr[covid_cases_aggr["country"] == country]
    country_cases = country_cases[country_cases["confirmed"] > 0]
    country_dates = pd.date_range(start=country_cases.last_update.min(), end=covid_cases_aggr.last_update.max())
    country_cases = country_cases.set_index('last_update').reindex(country_dates).fillna(method='pad').rename_axis('last_update').reset_index()
    country_cases = country_cases.reset_index()
    country_cases.rename(columns = {'index':'days_since_first_case'}, inplace = True) 
    country_cases["type"] = "actuals"
    
    
    #x = (country_cases["days_since_first_case"]).to_numpy().reshape((-1, 1))
    #y = (country_cases["confirmed"]).to_numpy().reshape((-1, 1))
    x = np.array((country_cases["days_since_first_case"]))
    y = np.array((country_cases["confirmed"]))
    confirmed_cases_pred = np.polyfit(x, np.log(y), 1)
    
    population = int((country_cases["population"]).tail(1))
    death_rate = float((country_cases["deaths"].tail(1) * 1.0000) / (country_cases["confirmed"].tail(1) * 1.0000))
    for i in range(country_cases["days_since_first_case"].max() + 1 , country_cases["days_since_first_case"].max() + days_to_forecast - 1):
        confirmed = np.exp(confirmed_cases_pred[1]) * np.exp(confirmed_cases_pred[0] * i) 
        deaths = confirmed * death_rate
        row = {
            "days_since_first_case": i,
            "last_update": (country_cases["last_update"].min() + datetime.timedelta(days=i)),
            "country": country,
            "population": population,
            "confirmed": round(confirmed),
            "deaths": round(deaths),
            "type": "forecast"
        }
        country_cases = country_cases.append(row, ignore_index=True)
        #print(predicted_row)
        #y_pred = np.exp(curve_fit[1]) * np.exp(curve_fit[0] * i)
        #print(y_pred)
        #print(row)
    
    #print(country_cases)
    
    #print(curve_fit[0], curve_fit[1])
    #
    #print(y_pred, y)
    li.append(country_cases)

covid_cases_final = pd.concat(li, axis=0, ignore_index=True)

In [13]:
## Add rate columns
covid_cases_final["cases_per_million"] = covid_cases_final["confirmed"] / (covid_cases_final["population"] / 1000000)
covid_cases_final["fatalities_per_million"] = covid_cases_final["deaths"] / (covid_cases_final["population"] / 1000000)
covid_cases_final["fatalities_per_cases"] = covid_cases_final["deaths"] / covid_cases_final["confirmed"]
#print((covid_cases_final.tail(1))[["days_since_first_case", "last_update", "country", "confirmed", "deaths"]])
#print(covid_cases_final.tail(6))


In [14]:
plt.figure(num=None, figsize=(18, 10), dpi=640, facecolor='w', edgecolor='k')
plt.xlabel('Days Since First Confirmed Case')
plt.ylabel('Cases Per Million')
plt.title('COVID-19 Country Curves Comparison')

comparing_country = "Brazil"
green_legend = mpatches.Patch(color='green', label=comparing_country)
green_legend_forecast = mpatches.Patch(color='green', label=comparing_country + " (forecast)", linestyle = ':')
comparing_country = covid_cases_final[covid_cases_final["country"] == comparing_country]
point_in_time = comparing_country["days_since_first_case"].max()
plt.plot('days_since_first_case', 'cases_per_million', data=comparing_country[comparing_country["type"] == "actuals"], linewidth=2, color="green")
plt.plot('days_since_first_case', 'cases_per_million', data=comparing_country[comparing_country["type"] == "forecast"], linewidth=2, color="green", linestyle = ':')


comparing_country = "Italy"
red_legend = mpatches.Patch(color='red', label=comparing_country)
comparing_country = covid_cases_final[covid_cases_final["country"] == comparing_country]
comparing_country = comparing_country[comparing_country["days_since_first_case"] <= point_in_time]
plt.plot('days_since_first_case', 'cases_per_million', data=comparing_country[comparing_country["type"] == "actuals"], linewidth=2, color="red")
plt.plot('days_since_first_case', 'cases_per_million', data=comparing_country[comparing_country["type"] == "forecast"], linewidth=2, color="red", linestyle = ':')

comparing_country = "United States"
blue_legend = mpatches.Patch(color='blue', label=comparing_country)
comparing_country = covid_cases_final[covid_cases_final["country"] == comparing_country]
comparing_country = comparing_country[comparing_country["days_since_first_case"] <= point_in_time]
plt.plot('days_since_first_case', 'cases_per_million', data=comparing_country[comparing_country["type"] == "actuals"], linewidth=2, color="blue")
plt.plot('days_since_first_case', 'cases_per_million', data=comparing_country[comparing_country["type"] == "forecast"], linewidth=2, color="blue", linestyle = ':')

comparing_country = "South Korea"
black_legend = mpatches.Patch(color='black', label=comparing_country)
comparing_country = covid_cases_final[covid_cases_final["country"] == comparing_country]
comparing_country = comparing_country[comparing_country["days_since_first_case"] <= point_in_time]
plt.plot('days_since_first_case', 'cases_per_million', data=comparing_country[comparing_country["type"] == "actuals"], linewidth=2, color="black")
plt.plot('days_since_first_case', 'cases_per_million', data=comparing_country[comparing_country["type"] == "forecast"], linewidth=2, color="black", linestyle = ':')


plt.legend(handles=[green_legend, green_legend_forecast, red_legend, blue_legend, black_legend])

plt.show()
#print(comparing_country_1)